### 0. 법정동코드(행정동코드)

In [1]:
import os
import numpy as np
print(os.getcwd())  

c:\HIuniv_Project-hyeonseo\notebooks


In [1]:
# 행정구역별 코드를 구하기 위해 행정동 코드 csv 파일을 불러온다.
import pandas as pd
csv_path = r'../data/raw/KIKcd_H.20250714_processed.xlsx'
cd = pd.read_excel(csv_path)
cd.head()

# 현재 법정동 코드 데이터 안에는 NaN값이 존재, 또한 이후에 total 데이터와 
# 읍면동 이름 불일치 문제등을 야기할 수 있음, 따라서 이후 total 데이터와 함께 정규화 진행

,행정동코드,시도명,시군구명,읍면동명,생성일자,말소일자,읍면동명_전처리
0,1100000000,서울특별시,NaN,NaN,19880423,NaN,NaN
1,1111000000,서울특별시,종로구,NaN,19880423,NaN,NaN
2,1111051000,서울특별시,종로구,청운동,19880423,20081101.0,청운동
3,1111051500,서울특별시,종로구,청운효자동,20081101,NaN,청운효자동
4,1111052000,서울특별시,종로구,효자동,19880423,20081101.0,효자동


### 1. 행정구역별 15세 미만 인구수

In [2]:
# 읍면동 별 아동 인구수 데이터를 불러온다.
pop15 = pd.read_excel(r'../data/raw/읍면동_15세미만.xlsx')

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
pop15.head()

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
1,서울특별시,소계,소계,15세미만,807110
2,NaN,종로구,소계,15세미만,9276
3,NaN,NaN,사직동,15세미만,619
4,NaN,NaN,삼청동,15세미만,130


In [4]:
# 첫 번째 행을 컬럼 헤더로 설정
pop15.columns = pop15.iloc[0]

# 첫 번째 행 삭제
pop15 = pop15[1:].reset_index(drop=True)

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,소계,소계,15세미만,807110
1,NaN,종로구,소계,15세미만,9276
2,NaN,NaN,사직동,15세미만,619
3,NaN,NaN,삼청동,15세미만,130
4,NaN,NaN,부암동,15세미만,683


In [5]:
# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
pop15['행정구역별(읍면동)(1)'] = pop15['행정구역별(읍면동)(1)'].ffill()
pop15['행정구역별(읍면동)(2)'] = pop15['행정구역별(읍면동)(2)'].ffill()
pop15['행정구역별(읍면동)(3)'] = pop15['행정구역별(읍면동)(3)'].ffill()

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,소계,소계,15세미만,807110
1,서울특별시,종로구,소계,15세미만,9276
2,서울특별시,종로구,사직동,15세미만,619
3,서울특별시,종로구,삼청동,15세미만,130
4,서울특별시,종로구,부암동,15세미만,683


In [6]:
# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
pop15 = pop15[pop15['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,종로구,사직동,15세미만,619
1,서울특별시,종로구,삼청동,15세미만,130
2,서울특별시,종로구,부암동,15세미만,683
3,서울특별시,종로구,평창동,15세미만,1484
4,서울특별시,종로구,무악동,15세미만,903


In [7]:
# 컬럼 이름 변경
pop15 = pop15.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

display(pop15.head())

,시도명,시군구명,읍면동명,연령별(1),총인구(명)
0,서울특별시,종로구,사직동,15세미만,619
1,서울특별시,종로구,삼청동,15세미만,130
2,서울특별시,종로구,부암동,15세미만,683
3,서울특별시,종로구,평창동,15세미만,1484
4,서울특별시,종로구,무악동,15세미만,903


In [8]:
# '연령별(1)' 컬럼 삭제
pop15 = pop15.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '아동인구수'로 변경
pop15 = pop15.rename(columns={'총인구(명)': '아동인구수'})

display(pop15.head())

,시도명,시군구명,읍면동명,아동인구수
0,서울특별시,종로구,사직동,619
1,서울특별시,종로구,삼청동,130
2,서울특별시,종로구,부암동,683
3,서울특별시,종로구,평창동,1484
4,서울특별시,종로구,무악동,903


In [9]:
# '아동인구수' 컬럼의 데이터 타입을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
pop15['아동인구수'] = pd.to_numeric(pop15['아동인구수'], errors='coerce')

# '아동인구수' 컬럼을 nullable integer 타입으로 변환 (NaN 포함 가능)
pop15['아동인구수'] = pop15['아동인구수'].astype('Int64')

# NaN값은 0으로 대체
pop15['아동인구수'] = pop15['아동인구수'].fillna(0).astype('Int64')

display(pop15.head())
pop15.isnull().sum()

,시도명,시군구명,읍면동명,아동인구수
0,서울특별시,종로구,사직동,619
1,서울특별시,종로구,삼청동,130
2,서울특별시,종로구,부암동,683
3,서울특별시,종로구,평창동,1484
4,서울특별시,종로구,무악동,903


0
시도명      0
시군구명     0
읍면동명     0
아동인구수    0
dtype: int64

In [10]:
pop15.query('읍면동명 == "일광읍"')

,시도명,시군구명,읍면동명,아동인구수
627,부산광역시,기장군,일광읍,4796


### 2. 행정구역별 65세 이상 인구수


In [11]:
# 읍면동 별 고령자 인구수 데이터를 불러온다.
pop65 = pd.read_excel(r'../data/raw/읍면동_65세이상.xlsx')
pop65.head()

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
1,서울특별시,소계,소계,65세이상,1791491
2,NaN,종로구,소계,65세이상,28376
3,NaN,NaN,사직동,65세이상,1826
4,NaN,NaN,삼청동,65세이상,577


In [12]:
# 첫 번째 행을 컬럼 헤더로 설정
pop65.columns = pop65.iloc[0]

# 첫 번째 행 삭제
pop65 = pop65[1:].reset_index(drop=True)

# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
pop65['행정구역별(읍면동)(1)'] = pop65['행정구역별(읍면동)(1)'].ffill()
pop65['행정구역별(읍면동)(2)'] = pop65['행정구역별(읍면동)(2)'].ffill()
pop65['행정구역별(읍면동)(3)'] = pop65['행정구역별(읍면동)(3)'].ffill()

# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
pop65 = pop65[pop65['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

# 컬럼 이름 변경
pop65 = pop65.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

# '연령별(1)' 컬럼 삭제
pop65 = pop65.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '고령자인구수'로 변경
pop65 = pop65.rename(columns={'총인구(명)': '고령자인구수'})

display(pop65.head())

,시도명,시군구명,읍면동명,고령자인구수
0,서울특별시,종로구,사직동,1826
1,서울특별시,종로구,삼청동,577
2,서울특별시,종로구,부암동,1953
3,서울특별시,종로구,평창동,3854
4,서울특별시,종로구,무악동,1492


In [13]:
# '고령자인구수' 컬럼의 데이터 타입을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
pop65['고령자인구수'] = pd.to_numeric(pop65['고령자인구수'], errors='coerce')

# 필요하다면 NaN 값을 처리 (예: 0으로 채우거나 해당 행 삭제) - 여기서는 NaN을 그대로 두고 nullable integer로 변환
# '고령자인구수' 컬럼을 nullable integer 타입으로 변환 (NaN 포함 가능)
pop65['고령자인구수'] = pop65['고령자인구수'].astype('Int64')

# NaN값은 0으로 대체
pop65['고령자인구수'] = pop65['고령자인구수'].fillna(0).astype('Int64')

display(pop65.head())
display(pop65.isnull().sum())

,시도명,시군구명,읍면동명,고령자인구수
0,서울특별시,종로구,사직동,1826
1,서울특별시,종로구,삼청동,577
2,서울특별시,종로구,부암동,1953
3,서울특별시,종로구,평창동,3854
4,서울특별시,종로구,무악동,1492


0
시도명       0
시군구명      0
읍면동명      0
고령자인구수    0
dtype: int64

### 3. 행정구역별 외국인 인구수 (미숙한 한국어는 재난 발생시에 위험을 높힐 수 있기에 점수 계산에 포함)

In [14]:
popFo = pd.read_excel(r'../data/raw/읍면동_총인구수.xlsx')
popFo.head()

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024,2024.1
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명),내국인(명)
1,서울특별시,소계,소계,합계,9335444,8968153
2,NaN,종로구,소계,합계,144486,132177
3,NaN,NaN,사직동,합계,8411,8138
4,NaN,NaN,삼청동,합계,2060,1994


In [15]:
# 첫 번째 행을 컬럼 헤더로 설정
popFo.columns = popFo.iloc[0]

# 첫 번째 행 삭제
popFo = popFo[1:].reset_index(drop=True)

# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
popFo['행정구역별(읍면동)(1)'] = popFo['행정구역별(읍면동)(1)'].ffill()
popFo['행정구역별(읍면동)(2)'] = popFo['행정구역별(읍면동)(2)'].ffill()
popFo['행정구역별(읍면동)(3)'] = popFo['행정구역별(읍면동)(3)'].ffill()

In [16]:
# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
popFo = popFo[popFo['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

# 컬럼 이름 변경
popFo = popFo.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

# '연령별(1)' 컬럼 삭제
popFo = popFo.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '고령자인구수'로 변경
popFo = popFo.rename(columns={'총인구(명)': '총인구', '내국인(명)' : '내국인'})

display(popFo.head())

,시도명,시군구명,읍면동명,총인구,내국인
0,서울특별시,종로구,사직동,8411,8138
1,서울특별시,종로구,삼청동,2060,1994
2,서울특별시,종로구,부암동,9057,8712
3,서울특별시,종로구,평창동,16392,16098
4,서울특별시,종로구,무악동,7446,7398


In [17]:
# '총인구'와 '내국인' 컬럼을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
# '내국인' 컬럼이 존재하는 경우에만 변환
if '내국인' in popFo.columns:
    popFo['내국인'] = pd.to_numeric(popFo['내국인'], errors='coerce')
popFo['총인구'] = pd.to_numeric(popFo['총인구'], errors='coerce')


# '총인구' - '내국인'으로 '외국인' 컬럼 생성
# '내국인' 컬럼이 존재하는 경우에만 뺄셈 수행, 아니면 NaN으로 초기화
if '내국인' in popFo.columns:
    popFo['외국인'] = popFo['총인구'] - popFo['내국인']
else:
    popFo['외국인'] = pd.NA # Or 0, depending on desired behavior when '내국인' is missing


# '총인구'와 '외국인' 컬럼의 NaN 값을 0으로 채우고 정수형으로 변환
popFo['총인구'] = popFo['총인구'].fillna(0).astype(int)
popFo['외국인'] = popFo['외국인'].fillna(0).astype(int)


# '내국인' 컬럼 삭제
if '내국인' in popFo.columns:
    popFo = popFo.drop(columns=['내국인'])

display(popFo.head())
display(popFo.isnull().sum())

,시도명,시군구명,읍면동명,총인구,외국인
0,서울특별시,종로구,사직동,8411,273
1,서울특별시,종로구,삼청동,2060,66
2,서울특별시,종로구,부암동,9057,345
3,서울특별시,종로구,평창동,16392,294
4,서울특별시,종로구,무악동,7446,48


0
시도명     0
시군구명    0
읍면동명    0
총인구     0
외국인     0
dtype: int64

### 4. 행정구역별 장애인 인구수

In [18]:
# 시군구 별 장애인 인구수 데이터를 불러온다.
popDP = pd.read_excel(r'../data/raw/시군구_장애인인구수.xlsx')

In [19]:
popDP.head()

,시군구별(1),시군구별(2),2024
0,시군구별(1),시군구별(2),합계
1,시군구별(1),시군구별(2),소계
2,전국,소계,2631356
3,서울특별시,소계,386316
4,NaN,종로구,5622


In [20]:
# 첫 번째 행을 컬럼 헤더로 설정
popDP.columns = popDP.iloc[1]

# 첫 번째 행 삭제
popDP = popDP[2:].reset_index(drop=True)

display(popDP.head())

1,시군구별(1),시군구별(2),소계
0,전국,소계,2631356
1,서울특별시,소계,386316
2,NaN,종로구,5622
3,NaN,중구,5317
4,NaN,용산구,6871


In [21]:
# 컬럼 이름 변경
popDP = popDP.rename(columns={'시군구별(1)': '시도명', '시군구별(2)': '시군구명', '소계': '장애인인구수'})

display(popDP.head())

1,시도명,시군구명,장애인인구수
0,전국,소계,2631356
1,서울특별시,소계,386316
2,NaN,종로구,5622
3,NaN,중구,5317
4,NaN,용산구,6871


In [22]:
popDP['시도명'] = popDP['시도명'].ffill()
popDP

1,시도명,시군구명,장애인인구수
0,전국,소계,2631356
1,서울특별시,소계,386316
2,서울특별시,종로구,5622
3,서울특별시,중구,5317
4,서울특별시,용산구,6871
...,...,...,...
264,경상남도,거창군,4958
265,경상남도,합천군,4489
266,제주특별자치도,소계,36818
267,제주특별자치도,제주시,25506


In [23]:
# 시군구명 컬럼의 값이 '소계'인 행들 삭제
popDP = popDP[popDP['시군구명'] != '소계']

display(popDP.head())

1,시도명,시군구명,장애인인구수
2,서울특별시,종로구,5622
3,서울특별시,중구,5317
4,서울특별시,용산구,6871
5,서울특별시,성동구,10743
6,서울특별시,광진구,12023


In [24]:
import pandas as pd, re
popDP['장애인인구수'] = pd.to_numeric(popDP['장애인인구수'], errors='coerce').astype('Int64')

# 1) 시군구명 → 구명만 추출 ------------------------------------------------------
def to_gu(name:str) -> str:
    if pd.isna(name): return name
    s = re.sub(r'\s+', '', str(name))         # 공백 제거
    return re.sub(r'.*시', '', s) if s.endswith('구') else s   # ‘○○시’ 앞부분 날림

popDP['시군구명'] = popDP['시군구명'].apply(to_gu)
popDP

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15540\2574171319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popDP['장애인인구수'] = pd.to_numeric(popDP['장애인인구수'], errors='coerce').astype('Int64')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15540\2574171319.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popDP['시군구명'] = popDP['시군구명'].apply(to_gu)


1,시도명,시군구명,장애인인구수
2,서울특별시,종로구,5622
3,서울특별시,중구,5317
4,서울특별시,용산구,6871
5,서울특별시,성동구,10743
6,서울특별시,광진구,12023
...,...,...,...
263,경상남도,함양군,3574
264,경상남도,거창군,4958
265,경상남도,합천군,4489
267,제주특별자치도,제주시,25506


In [25]:
total_DP = popDP['장애인인구수'].sum()
total_DP

2631356

### 5. 전체 데이터들 merge

In [26]:
total = pd.merge(pop15, pop65, on=['시도명', '시군구명', '읍면동명'], how='inner')
total = pd.merge(total, popFo, on=['시도명', '시군구명', '읍면동명'], how='inner')
total = pd.merge(total, popDP, on=['시도명', '시군구명'], how='left')
display(total.head())

,시도명,시군구명,읍면동명,아동인구수,고령자인구수,총인구,외국인,장애인인구수
0,서울특별시,종로구,사직동,619,1826,8411,273,5622
1,서울특별시,종로구,삼청동,130,577,2060,66,5622
2,서울특별시,종로구,부암동,683,1953,9057,345,5622
3,서울특별시,종로구,평창동,1484,3854,16392,294,5622
4,서울특별시,종로구,무악동,903,1492,7446,48,5622


In [27]:
# 광명1동의 경우, 통계청 조사에 포함되지 않아서 0이 뜸, 이 후계산에 NaN 값 발생할 수 있으므로 아예 삭제
total = total[total['총인구'] != 0].copy()

In [28]:
total['시도명'] = total['시도명'].replace('전북특별자치도', '전라북도')
total['시도명'] = total['시도명'].replace('강원특별자치도', '강원도')

### 6. 법정동 코드 merge

In [29]:

def norm_gu(name):
    if pd.isna(name) or name == '':       # 빈 문자열 방지
        return ''
    return str(name).split()[-1]

def norm_dong(name):
    if pd.isna(name): 
        return name
    s = str(name).strip()
    # 창제1동, 당감제1동 등: '제숫자' → '숫자'
    s = re.sub(r'제(\d+)', r'\1', s)
    # 구분 기호 통일 후 제거
    s = s.replace('.', '·').replace('·', '').replace('ㆍ','')
    return s

cd.loc[(cd['시도명']=='세종특별자치시') & (cd['시군구명'].isna()), '시군구명'] = '세종시'
cd['시군구명'] = cd['시군구명'].fillna('')
cd['key_gu']   = cd['시군구명'].apply(norm_gu)
total['key_gu'] = total['시군구명'].apply(norm_gu)

cd['key_dong']   = cd['읍면동명'].apply(norm_dong)
total['key_dong'] = total['읍면동명'].apply(norm_dong)

cd_sorted = (cd.sort_values(['말소일자','생성일자'])   # 말소일자 NaN 이 먼저 옴
                 .drop_duplicates(['시도명','key_gu','key_dong'],
                                   keep='last'))

cd_subset = cd_sorted[['시도명','key_gu','key_dong','행정동코드']]

total = (total
         .merge(cd_subset,
                left_on = ['시도명','key_gu','key_dong'],
                right_on=['시도명','key_gu','key_dong'],
                how='left')
         .drop(columns=['key_gu','key_dong'])      # 정규화용 컬럼 제거
        )


# 6) 결과 확인 -----------------------------------------------------------
print(total.shape)                   # (3554, 원하는 컬럼 수)
print(total['행정동코드'].isna().sum())  # 0 → 전부 매핑 완료

(3553, 9)
0


In [30]:
total.isnull().sum()

시도명       0
시군구명      0
읍면동명      0
아동인구수     0
고령자인구수    0
총인구       0
외국인       0
장애인인구수    0
행정동코드     0
dtype: int64

## 사회취약계층 계산

In [31]:
cols = ['아동인구수','고령자인구수','외국인']
for c in cols:
    total[c+'_pct'] = total[c] / total['총인구'] * 100   

In [32]:
total['장애인_pct'] = (total['장애인인구수'] / total_DP) * 100
total

,시도명,시군구명,읍면동명,아동인구수,고령자인구수,총인구,외국인,장애인인구수,행정동코드,아동인구수_pct,고령자인구수_pct,외국인_pct,장애인_pct
0,서울특별시,종로구,사직동,619,1826,8411,273,5622,1111053000,7.35941,21.709666,3.245750,0.213654
1,서울특별시,종로구,삼청동,130,577,2060,66,5622,1111054000,6.31068,28.009709,3.203883,0.213654
2,서울특별시,종로구,부암동,683,1953,9057,345,5622,1111055000,7.541128,21.563432,3.809208,0.213654
3,서울특별시,종로구,평창동,1484,3854,16392,294,5622,1111056000,9.053197,23.511469,1.793558,0.213654
4,서울특별시,종로구,무악동,903,1492,7446,48,5622,1111057000,12.127317,20.037604,0.644641,0.213654
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3548,제주특별자치도,서귀포시,서홍동,1810,1731,10804,267,11312,5013058000,16.753054,16.021844,2.471307,0.429892
3549,제주특별자치도,서귀포시,대륜동,2010,2852,14715,316,11312,5013059000,13.659531,19.381583,2.147469,0.429892
3550,제주특별자치도,서귀포시,대천동,2074,2331,13438,392,11312,5013060000,15.433844,17.346331,2.917101,0.429892
3551,제주특별자치도,서귀포시,중문동,1461,2196,12053,467,11312,5013061000,12.121464,18.21953,3.874554,0.429892


배민기·오후 (2021) – 계층분석과정(AHP)을 이용한 감염병 취약지표 가중치 평가 
- 이를 참조하여 사회취약계층 지수 산출

1. 65세 이상 고령자 0.30
2. 아동 0.25
3. 장애인 0.35 
4. 외국인 0.10

In [33]:
tmp = (total['고령자인구수_pct'] * 0.30) + (total['아동인구수_pct'] * 0.25) + (total['장애인_pct'] * 0.35) + (total['외국인_pct']*0.1)
vmin = tmp.min()
vmax = tmp.max()


사회취약지수 = (
    (tmp - vmin) / (vmax - vmin)   # 0~1
    * 100                                         # 0~100
)
사회취약지수

0       31.001083
1       41.566627
2       31.377889
3       36.329058
4       33.800833
          ...    
3548    35.167733
3549    36.483232
3550    35.897353
3551     32.83638
3552    41.454539
Length: 3553, dtype: Float64

In [34]:
사회취약지수.describe()

count       3553.0
mean     46.079437
std      20.489119
min            0.0
25%      29.679346
50%      39.757358
75%      63.931931
max          100.0
dtype: Float64

In [35]:
total['사회취약지수'] = 사회취약지수
print(total.shape)
print(total.head())
print(total.isnull().sum())

(3553, 14)
     시도명 시군구명 읍면동명  아동인구수  고령자인구수    총인구  외국인  장애인인구수       행정동코드  아동인구수_pct  \
0  서울특별시  종로구  사직동    619    1826   8411  273    5622  1111053000    7.35941   
1  서울특별시  종로구  삼청동    130     577   2060   66    5622  1111054000    6.31068   
2  서울특별시  종로구  부암동    683    1953   9057  345    5622  1111055000   7.541128   
3  서울특별시  종로구  평창동   1484    3854  16392  294    5622  1111056000   9.053197   
4  서울특별시  종로구  무악동    903    1492   7446   48    5622  1111057000  12.127317   

   고령자인구수_pct   외국인_pct   장애인_pct     사회취약지수  
0   21.709666  3.245750  0.213654  31.001083  
1   28.009709  3.203883  0.213654  41.566627  
2   21.563432  3.809208  0.213654  31.377889  
3   23.511469  1.793558  0.213654  36.329058  
4   20.037604  0.644641  0.213654  33.800833  
시도명           0
시군구명          0
읍면동명          0
아동인구수         0
고령자인구수        0
총인구           0
외국인           0
장애인인구수        0
행정동코드         0
아동인구수_pct     0
고령자인구수_pct    0
외국인_pct       0
장애인_pct       0
사회취약지수        0
dt

## 전처리된 데이터 export

In [36]:
from pathlib import Path

out_dir = Path.cwd().parent / 'data' / 'processed'   # notebooks → .. → data/processed
out_dir.mkdir(parents=True, exist_ok=True)           # 폴더 없으면 생성

total.to_csv(
    out_dir / '202506_읍면동_사회취약계층표.csv',
    index=False,
    encoding='utf-8-sig'     # ← BOM 포함 UTF‑8 저장
)

In [37]:
사회취약지수표 = total[['시도명','시군구명','읍면동명','행정동코드','사회취약지수']]

out_dir = Path.cwd().parent / 'data' / 'processed'
out_dir.mkdir(parents=True, exist_ok=True)

# UTF‑8 + BOM → Excel이 UTF‑8 인코딩으로 자동 인식
사회취약지수표.to_csv(
    out_dir / '사회취약지수표.csv',
    index=False,
    encoding='utf-8-sig'
)

In [38]:
total["행정동코드"] = (
    total["행정동코드"]
        .astype(str)                 # 3611025000  or  3611025000.0
        .str.split(".").str[0]        # 3611025000  (소수점 제거)
        .str.zfill(10)                # 10자리 0패딩 (0012312345 같은 코드 대비)
        .str.strip()                  # 혹시 모를 공백 제거
)

# folium을 통한 지도 시각화

In [40]:
import json, io, base64  # JSON 처리, 메모리 I/O, Base64 인코딩/디코딩(사용처는 외부일 수 있음)
import numpy as np  # 수치 계산 (NaN 처리, 비교 등)
import pandas as pd  # 테이블형 데이터 처리
import matplotlib.pyplot as plt  # 시각화 (현재는 폰트 설정에 사용)
import matplotlib as mpl  # matplotlib 전역 설정 조작
import matplotlib.font_manager as fm  # 시스템 폰트 탐색
import folium  # 지도 시각화 라이브러리
from folium.features import GeoJsonTooltip  # GeoJSON에 툴팁 붙일 때
from folium.plugins import FloatImage  # 지도 위에 이미지 띄우는 플러그인 (현재 직접 사용되진 않음)

# ───────────────────────────────────────────────
# 0) 한글 폰트 / 마이너스 깨짐 방지
# ───────────────────────────────────────────────
# 우선순위로 사용할 한글 폰트 목록 정의하고 시스템에 있으면 설정
preferred = ['Malgun Gothic', 'AppleGothic', 'NanumGothic']
for f in preferred:
    if any(ft.name == f for ft in fm.fontManager.ttflist):
        mpl.rcParams['font.family'] = f  # 첫 번째 발견된 폰트로 전체 폰트 설정
        break
# 음수 기호가 깨지는 것을 방지
mpl.rcParams['axes.unicode_minus'] = False

# ───────────────────────────────────────────────
# 1) GeoJSON 17개 로드 & 결합
# ───────────────────────────────────────────────
# 시도별로 분리된 행정동 GeoJSON 파일 경로 정의
paths = {
    '세종': '../data/raw/hangjeongdong_세종특별자치시.geojson',
    '경북': '../data/raw/hangjeongdong_경상북도.geojson',
    '강원': '../data/raw/hangjeongdong_강원도.geojson',
    '경기': '../data/raw/hangjeongdong_경기도.geojson',
    '경남': '../data/raw/hangjeongdong_경상남도.geojson',
    '광주': '../data/raw/hangjeongdong_광주광역시.geojson',
    '대전': '../data/raw/hangjeongdong_대전광역시.geojson',
    '대구': '../data/raw/hangjeongdong_대구광역시.geojson',
    '서울': '../data/raw/hangjeongdong_서울특별시.geojson',
    '부산': '../data/raw/hangjeongdong_부산광역시.geojson',
    '인천': '../data/raw/hangjeongdong_인천광역시.geojson',
    '울산': '../data/raw/hangjeongdong_울산광역시.geojson',
    '제주': '../data/raw/hangjeongdong_제주특별자치도.geojson',
    '전북': '../data/raw/hangjeongdong_전라북도.geojson',
    '전남': '../data/raw/hangjeongdong_전라남도.geojson',
    '충남': '../data/raw/hangjeongdong_충청남도.geojson',
    '충북': '../data/raw/hangjeongdong_충청북도.geojson',
}

# 각 파일을 열어 GeoJSON 객체로 로드
geos = [json.load(open(p, encoding='utf-8')) for p in paths.values()]
# 모든 feature를 하나의 FeatureCollection으로 합쳐 전국 단위 GeoJSON 생성
geo_all = {"type": "FeatureCollection",
           "features": [f for g in geos for f in g['features']]}

# ───────────────────────────────────────────────
# 2) total DataFrame 준비
# ───────────────────────────────────────────────
# 외부에서 준비된 vulnerability_scores.csv를 불러와야 하는 부분 (주석 처리되어 실제 로딩은 이 바깥에서 이루어져야 함)
# total = pd.read_csv('../data/processed/vulnerability_scores.csv')
# columns: [행정동코드, 총인구, 고령자인구수, 사회취약지수]

# ───────────────────────────────────────────────
# 3) 취약등급 계산 & total 컬럼 추가
# ───────────────────────────────────────────────
# 점수 구간 기준 설정: 0~25, 25~50, 50~75, 75~100
bins = [0, 25, 50, 75, 100]

# 사회취약지수 값을 기반으로 취약등급(1~4) 계산하는 함수
def grade(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return 1  # 값이 없거나 NaN이면 가장 낮은 등급 부여
    for i in range(1, len(bins)):
        if x < bins[i]:
            return i  # 해당 구간에 맞는 등급 반환
    return len(bins) - 1  # 최대 등급

# total DataFrame에 계산된 등급 컬럼 추가
total['취약등급'] = total['사회취약지수'].apply(grade)

# 행정동 코드 → 이름 매핑 준비 (GeoJSON의 properties에서 가져옴)
code2name = {str(f['properties']['adm_cd2']): f['properties']['adm_nm']
             for f in geo_all['features']}
# total에 행정동명 컬럼 추가 (매핑 실패하면 NaN)
total['행정동명'] = total['행정동코드'].astype(str).map(code2name)

# total을 dict 형태로 만들어 빠른 조회용 사전 생성
data_dict = total.set_index('행정동코드').to_dict(orient='index')
# 매핑되지 않은 feature에 들어갈 기본값 (예: 데이터 없음)
empty = {'총인구': 19543, '고령자인구수': 0, '사회취약지수': 12, '취약등급': 1}

# ───────────────────────────────────────────────
# 4) GeoJSON 속성 병합
# ───────────────────────────────────────────────
# 모든 GeoJSON feature에 total의 값을 주입하여 속성 보강
for feat in geo_all['features']:
    c = str(feat['properties']['adm_cd2'])  # 행정동 코드 문자열화
    row = data_dict.get(c)
    if row:
        # 해당 코드가 total에 있으면 실제 값으로 덮어쓰기
        feat['properties'].update({
            '총인구': row['총인구'],
            '고령자인구수': row['고령자인구수'],
            '사회취약지수': row['사회취약지수'],
            '취약등급': row['취약등급'],
        })
    else:
        # 없으면 기본값 채우기
        feat['properties'].update(empty)

# ───────────────────────────────────────────────
# 5) 지도 & Choropleth
# ───────────────────────────────────────────────
# 기본 지도 생성: 중앙 위치, 줌 레벨, 라이트한 타일, 축척 표시
m = folium.Map(location=[36, 127], zoom_start=7,
               tiles='cartodbpositron', control_scale=True)

# Choropleth 레이어 추가: 사회취약지수를 컬러로 시각화
choropleth = folium.Choropleth(
    geo_data=geo_all,  # 병합된 전국 GeoJSON
    data=total,  # 데이터 프레임
    columns=('행정동코드', '사회취약지수'),  # 매핑 열
    key_on='feature.properties.adm_cd2',  # GeoJSON 측 속성 키
    fill_color='RdPu',  # 색상 팔레트
    fill_opacity=0.85, line_opacity=0.3, line_weight=0.4,  # 스타일
    nan_fill_color='#fde0dd',  # 결측 색
    bins=bins, highlight=True,  # 등급 구간 및 하이라이트 활성화
    name='사회취약지수(%)'  # 레이어 이름 (레이어 컨트롤에 표시)
).add_to(m)

# ───────────────────────────────────────────────
# 6) Tooltip (오류 수정: choropleth.geojson 에 직접 부착)
# ───────────────────────────────────────────────
# 마우스 오버 시 보여줄 필드 정의 (행정동명, 지수, 등급)
GeoJsonTooltip(
    fields=['adm_nm', '사회취약지수', '취약등급'],
    aliases=['행정동', '취약지수(%)', '취약등급'],
    localize=True,  # 숫자 등을 지역화
    sticky=False,  # 마우스 포인터에 고정하지 않음
    labels=True,  # 필드명 라벨 표시
    style=("background-color: rgba(255,255,255,0.9);"
           "border:1px solid #999;border-radius:3px;"
           "box-shadow:2px 2px 6px rgba(0,0,0,0.15);"
           "font-size:12px;padding:4px;")  # 툴팁 스타일
).add_to(choropleth.geojson)  # Choropleth의 내부 GeoJSON에 붙임



# ───────────────────────────────────────────────
# 6-1) Chart.js 패널(버튼 + 동적 업데이트) ―★ 새 버전
# ───────────────────────────────────────────────
from branca.element import Template, MacroElement, Element  # HTML/JS 임베디드용 브랜카 요소
import json  # Python 데이터를 JS로 직렬화할 때 사용

# (1) <head>에 Chart.js CDN - 중복 삽입 방지
# 이미 삽입되어 있지 않으면 script 태그로 Chart.js 불러오기
if not any("https://cdn.jsdelivr.net/npm/chart.js" in s.render()
           for s in m.get_root().header._children.values()):
    m.get_root().header.add_child(
        Element('<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>')
    )

# (2) ‘전국’ + 17개 시‧도 top-10 데이터 준비
def sido_of(name):
    """
    행정동명에서 시도명 추출 (paths의 키와 prefix 비교)
    매칭이 안 되면 '기타' 반환
    """
    if not isinstance(name, str):
        return '기타'
    for k in paths.keys():
        if name.startswith(k):
            return k
    return '기타'

province_data = {}
# 전국 기준 상위 10개 행정동 추출
nat_top = total.nlargest(10, '사회취약지수')
province_data['전국'] = {
    'labels': nat_top['행정동명'].tolist(),
    'values': nat_top['사회취약지수'].tolist()
}

# 각 시도별로 상위 10개 추출하여 구조체에 저장
total['시도명'] = total['행정동명'].apply(sido_of)
for sido, grp in total.groupby('시도명'):
    top10 = grp.nlargest(10, '사회취약지수')
    province_data[sido] = {
        'labels': top10['행정동명'].tolist(),
        'values': top10['사회취약지수'].tolist()
    }

# (3) Python → JS 직렬화: 데이터를 템플릿 안 JS에서 쓸 수 있게 JSON 문자열로 변환
province_js = json.dumps(province_data, ensure_ascii=False)
# 버튼 HTML 생성: 각 시도/전국을 선택하는 칩 스타일 버튼 (단, '기타'는 제외)
btn_html = ''.join(
    f'<button class="sido-btn" data-sido="{k}">{k}</button>'
    for k in province_data.keys() if k != '기타'
)

# (4) Folium에 삽입할 매크로 템플릿 정의 (HTML + 스크립트)
template_str = f"""
{{% macro html(this, kwargs) %}}
<style>
  /* 버튼‧패널 스타일: 현대적 디자인 (라운드, 섀도우, 칩) */
  #vuln-panel {{
      position:absolute; top:200px; right:10px; z-index:9999;
      width:380px; background:#ffffff; padding:12px 14px;
      border-radius:12px; box-shadow:0 4px 16px rgba(0,0,0,.20);
      font-family:'Segoe UI','NanumGothic',sans-serif;
  }}
  #sido-buttons {{
      display:flex; flex-wrap:wrap; gap:8px; justify-content:center;
      margin-top:10px;
  }}
  .sido-btn {{
      padding:7px 16px; font-size:13px; border:none; border-radius:18px;
      background:#f6f6f7; color:#444; cursor:pointer;
      box-shadow:0 1px 2px rgba(0,0,0,.10);
      transition:background .25s ease, box-shadow .25s ease, color .25s;
  }}
  .sido-btn:hover {{
      background:#fae6ef; box-shadow:0 2px 6px rgba(0,0,0,.15);
  }}
  .sido-btn.active {{
      background:#fa9fb5; color:#fff;
      box-shadow:0 2px 8px rgba(0,0,0,.20);
  }}
</style>

<div id="vuln-panel">
  <canvas id="barChart" width="350" height="260"></canvas>  <!-- Top-10 수평 바차트 영역 -->
  <div id="sido-buttons">
    {btn_html}  <!-- 시도 선택 버튼들 삽입 (기타 제외) -->
  </div>
</div>
{{% endmacro %}}

{{% macro script(this, kwargs) %}}
/* ----- 데이터 & 차트 ----- */
const provinceData = {province_js};  // Python에서 직렬화한 지역별 top-10 데이터

const ctx   = document.getElementById('barChart').getContext('2d');
const chart = new Chart(ctx, {{
  type: 'bar',
  data: {{
    labels: provinceData['전국'].labels,
    datasets: [{{ data: provinceData['전국'].values,
                  backgroundColor:'#fa9fb5' }}]
  }},
  options: {{
    indexAxis:'y',  // 수평 막대
    scales:{{ x:{{ beginAtZero:true }} }},  // x축 0부터 시작
    plugins:{{
      legend:{{ display:false }},  // 범례 숨김
      tooltip:{{ callbacks:{{
        label:c=>c.parsed.x + '%'  // 툴팁에 % 붙여 표시
      }} }}
    }}
  }}
}});

/* ----- 버튼 이벤트 (active 토글 포함) ----- */
document.getElementById('sido-buttons').addEventListener('click', e => {{
  if(!e.target.classList.contains('sido-btn')) return;  // 버튼 아닌 클릭 무시

  // 모든 버튼에서 active 제거 후 클릭된 것만 활성화
  document.querySelectorAll('.sido-btn')
          .forEach(b=>b.classList.remove('active'));
  e.target.classList.add('active');

  // 선택한 시도에 맞춰 차트 데이터 교체
  const key   = e.target.dataset.sido;
  const pdata = provinceData[key] || provinceData['전국'];
  chart.data.labels = pdata.labels;
  chart.data.datasets[0].data = pdata.values;
  chart.update();
}});
{{% endmacro %}}
"""

# MacroElement로 만들어 지도에 붙임 (HTML+JS 삽입)
panel = MacroElement()
panel._template = Template(template_str)
m.get_root().add_child(panel)

# ───────────────────────────────────────────────
# 7) 레이어 컨트롤 + 저장 (변경 없음)
# ───────────────────────────────────────────────
# 사용자에게 레이어 토글 UI 제공
folium.LayerControl(collapsed=False).add_to(m)
# 결과물을 HTML 파일로 저장
m.save(r'../results/maps/korea_vulnerability_map.html')

